In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import colors
import matplotlib
import glob
import cPickle as pk
import os
from itertools import permutations, product
import multiprocessing as mp

plt.style.use('classic')
%matplotlib inline
data_dir = "./results/important_plots_and_data/"
plot_dir = "./ALLRUNS3/figures/"
matplotlib.use('TkAgg')


/Users/jonathanking/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/jonathanking/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
class Run(object):
    def __init__(self, params, paths):
        self.x = 0
        self.params = params
        self.FF = pk.load(open(paths[0], "r")) 
        self.FT = pk.load(open(paths[1], "r")) 
        self.TF = pk.load(open(paths[2], "r")) 
        self.TT = pk.load(open(paths[3], "r")) 
    def __repr__(self):
        mod = ""
        if "default" in self.params:
            mod = "defaults"
            return mod
        if "network" in self.params:
            return "Constrain Network"
        p = self.params.split("_")
        if "ps0.5"  in p or "ps0.999" in p:
            mod = "Prob Stay " + p[1].replace(".","")
        elif "if20"  in p or "if2"  in p:
            mod = "IR Time " + p[2]
        elif "rv1.0"  in p or "rv10.0"  in p:
            mod = "Rvalue "  + p[3][:-2]
        else:
            mod = "EI Time "  + p[4]
        return mod
    def title(self):
        mod = ""
        if "default" in self.params:
            mod = "Default Parameters"
            return mod
        if "network" in self.params:
            return "Constrain Network"
        p = self.params.split("_")
        if "ps0.5"  in p:
            mod = "$p_{stay} = 50\%$ "
        elif "ps0.999" in p:
            mod = "$p_{stay} = 99.9\%$"
        elif "if20"  in p:
            mod = "$T_{I\\rightarrow R} = 20$"
        elif "if2"  in p:
            mod = "$T_{I\\rightarrow R} = 2$"
        elif "rv1.0"  in p:
            mod = "$R_0 = 1$"
        elif "rv10.0" in p:
            mod = "$R_0 = 10$"
        elif "il50" in p:
            mod = "$T_{E\\rightarrow I} = 50$"
        else:
            mod = "$T_{E\\rightarrow I} = 5$"
        return mod
    def filename(self):
        return "_".join(str(self).split())
        
pickle_paths = glob.glob("ALLRUNS3/*/NODE*.pkl")
runs = []
i = 0
while i < len(pickle_paths) - 3:
    paths = pickle_paths[i], pickle_paths[i+1], pickle_paths[i+2], pickle_paths[i+3]
    r = Run(pickle_paths[i].split("/")[1], paths)
    runs.append(r)
    i += 4


In [3]:
print runs

[Prob Stay ps05, Prob Stay ps0999, IR Time if20, IR Time if2, Rvalue rv1, Rvalue rv10, EI Time il5, EI Time il50, defaults]


In [158]:
FILETYPE = ".png"
_DPI = 300
for r in runs:
    total_I = [np.sum([X[pair]['Ivec'] for pair in product(range(14), range(14))], axis=0) for X in [r.FF, r.TT, r.TF, r.FT]]
    total_S = [np.sum([X[pair]['Svec'] for pair in product(range(14), range(14))], axis=0) for X in [r.FF, r.TT, r.TF, r.FT]]
    total_E = [np.sum([X[pair]['Evec'] for pair in product(range(14), range(14))], axis=0) for X in [r.FF, r.TT, r.TF, r.FT]]
    total_R = [np.sum([X[pair]['Rvec'] for pair in product(range(14), range(14))], axis=0) for X in [r.FF, r.TT, r.TF, r.FT]]
    
    # Susceptible
    xs=np.arange(len(r.FF[pair]['Svec']))
    plt.plot(xs, total_S[0], label="FF")
    plt.plot(xs, total_S[1], label="TT")
    plt.plot(xs, total_S[2], label="TF")
    plt.plot(xs, total_S[3], label="FT")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Susceptible")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(r.title())
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
#     plt.ylim(0, 1600000)
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_S" + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
    
    
    # Infected
    xs=np.arange(len(r.FF[pair]['Ivec']))
    plt.plot(xs, total_I[0], label="FF")
    plt.plot(xs, total_I[1], label="TT")
    plt.plot(xs, total_I[2], label="TF")
    plt.plot(xs, total_I[3], label="FT")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Infected")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(r.title())
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
    if "$R_0 = 10$" not in r.title():
        plt.ylim(0, 8500)
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_I" + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
    
    # Removed
    xs=np.arange(len(r.FF[pair]['Rvec']))
    plt.plot(xs, total_R[0], label="FF")
    plt.plot(xs, total_R[1], label="TT")
    plt.plot(xs, total_R[2], label="TF")
    plt.plot(xs, total_R[3], label="FT")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Removed")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(r.title())
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
    plt.ylim(0, 32000)
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_R" + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
    

    # Exposed
    xs=np.arange(len(r.FF[pair]['Evec']))
    plt.plot(xs, total_E[0], label="FF")
    plt.plot(xs, total_E[1], label="TT")
    plt.plot(xs, total_E[2], label="TF")
    plt.plot(xs, total_E[3], label="FT")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Exposed")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(r.title())
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
    plt.ylim(0, 32000)
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_E" + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
    
    # I + R
    xs=np.arange(len(r.FF[pair]['Evec']))
    plt.plot(xs, total_I[0] + total_R[0], label="FF")
    plt.plot(xs, total_I[1] + total_R[1], label="TT")
    plt.plot(xs, total_I[2] + total_R[2], label="TF")
    plt.plot(xs, total_I[3] + total_R[3], label="FT")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Infected + Removed")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(r.title())
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
#     plt.ylim(0, 32000)
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_IR" + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
    
    
    

In [171]:
def getnodesums(r, nodestats, RUNSTR):
    s = nodestats[(0,0)]["Ivec"].shape
    quad1_I = np.zeros(s)
    quad1_O = np.zeros(s)
    quad2_I = np.zeros(s)
    quad2_O = np.zeros(s)
    quad3_I = np.zeros(s)
    quad3_O = np.zeros(s)
    quad4_I = np.zeros(s)
    quad4_O = np.zeros(s)
    
    for x in range(14):
        for y in range(14):
            pt = (x, y)
            if x <= 6 and y <= 6:
                quad1_I += nodestats[pt]["Ivec"]
                quad1_O += nodestats[pt]["Ivec"] + nodestats[pt]["Svec"] + nodestats[pt]["Evec"]
            elif x <= 6 and y >=6:
                quad2_I += nodestats[pt]["Ivec"]
                quad2_O += nodestats[pt]["Ivec"] + nodestats[pt]["Svec"] + nodestats[pt]["Evec"]
            elif x >= 6 and y <= 6:
                quad3_I += nodestats[pt]["Ivec"]
                quad3_O += nodestats[pt]["Ivec"] + nodestats[pt]["Svec"] + nodestats[pt]["Evec"]
            elif x >=6 and y >= 6:
                quad4_I += nodestats[pt]["Ivec"]
                quad4_O += nodestats[pt]["Ivec"] + nodestats[pt]["Svec"] + nodestats[pt]["Evec"]
    
    quad1_P = quad1_I * 100 / quad1_O
    quad2_P = quad2_I * 100 / quad2_O
    quad3_P = quad3_I * 100 / quad3_O
    quad4_P = quad4_I * 100 / quad4_O
    ZOOM = 400
    
    xs=np.arange(len(nodestats[pt]["Ivec"]))
    plt.plot(xs, quad1_P, label="Quad1")
    plt.plot(xs, quad2_P, label="Quad2")
    plt.plot(xs, quad3_P, label="Quad3")
    plt.plot(xs, quad4_P, label="Quad4")
#     plt.plot(xs[:ZOOM], quad1_P[:ZOOM], label="Q1")
#     plt.plot(xs[:ZOOM], quad2_P[:ZOOM], label="Q2")
#     plt.plot(xs[:ZOOM], quad3_P[:ZOOM], label="Q3")
#     plt.plot(xs[:ZOOM], quad4_P[:ZOOM], label="Q4")
#     plt.ylim(0, 0.0037)
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=4, mode="expand", borderaxespad=0.)
    plt.xlabel("Timestep")
    plt.ylabel("Percent Infected")
    plt.gcf().subplots_adjust(left=0.2, top=.81, bottom = 0.14)
    plt.title(" ".join(str(r).split()) + " " + RUNSTR + ": Percent Infected" )
    ax = plt.gca()
    ttl = ax.title
    ttl.set_position([.5, 1.15])
    
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_QUAD" + RUNSTR + FILETYPE, dpi=_DPI,pad_inches=0.5)
    plt.close()
                

In [172]:
FILETYPE = ".png"
_DPI = 300
for r in runs[::-1]:
    getnodesums(r, r.TT, "TT")
    getnodesums(r, r.TF, "TF")
    getnodesums(r, r.FT, "FT")
    getnodesums(r, r.FF, "FF")
    

/Users/jonathanking/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: RuntimeWarning: invalid value encountered in divide
/Users/jonathanking/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in divide
/Users/jonathanking/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in divide
/Users/jonathanking/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in divide


In [6]:
gif_dir = "./ALLRUNS3/gifs/"

for i in [0]:
    r = runs[i]
    for constraint in ["FF"]:
        print "Generating", r, constraint, "..."
#         print zip(inputs, [constraint for x in range(len(inputs))])
#         pool.map(processInput, zip(inputs, [constraint for x in range(len(inputs))]))
        for t in range(0, 3000, 1):
            if os.path.isfile(gif_dir + r.filename() + "/" + constraint + "/{0:04}.png".format(t)):
                continue
            xs = []
            ys = []
            cs = []
            for x in range(14):
                for y in range(14):
                    xs.append(x)
                    ys.append(y)
                    # I + R
                    # interested = abs(r.FF[(x,y)]["Ivec"][t]) + abs(r.FF[(x,y)]["Rvec"][t])
                    # I only
                    if constraint == "TT":
                        interested = abs(r.TT[(x,y)]["Ivec"][t])
                        other = sum(map(abs, [r.TT[(x,y)]["Ivec"][t], r.TT[(x,y)]["Svec"][t], r.TT[(x,y)]["Rvec"][t] , r.TT[(x,y)]["Evec"][t]]))
                    elif constraint == "FF":
                        interested = abs(r.FF[(x,y)]["Ivec"][t])
                        other = sum(map(abs, [r.FF[(x,y)]["Ivec"][t], r.FF[(x,y)]["Svec"][t], r.FF[(x,y)]["Rvec"][t] , r.FF[(x,y)]["Evec"][t]]))
                    elif constraint == "TF":
                        interested = abs(r.TF[(x,y)]["Ivec"][t])
                        other = sum(map(abs, [r.TF[(x,y)]["Ivec"][t], r.TF[(x,y)]["Svec"][t] , r.TF[(x,y)]["Evec"][t]]))
                    cs.append(interested/ float(other))

            plt.scatter(xs, ys, c=cs, cmap="Reds", s=100)
            plt.title(str(r) + " " + constraint + " , t = " + str(t))

            if not os.path.isdir(gif_dir + r.filename() + "/" + constraint):
                os.makedirs(gif_dir + r.filename() + "/" + constraint)

            plt.savefig(gif_dir + r.filename() + "/" + constraint + "/{0:04}.png".format(t))
            plt.close()
        

        

Generating Prob Stay ps05 FF ...


In [ ]:
def processInput(t_const):
    t, constraint = t_const
    print t, constraint
#     if os.path.isfile(gif_dir + r.filename() + "/" + constraint + "/{0:04}.png".format(t)):
#         return 
    xs = []
    ys = []
    cs = []
    for x in range(14):
        for y in range(14):
            xs.append(x)
            ys.append(y)
            # I only
            if constraint == "TT":
                interested = abs(r.TT[(x,y)]["Ivec"][t])
                other = sum(map(abs, [r.TT[(x,y)]["Ivec"][t], r.TT[(x,y)]["Svec"][t], r.TT[(x,y)]["Rvec"][t] , r.TT[(x,y)]["Evec"][t]]))
            elif constraint == "FF":
                interested = abs(r.FF[(x,y)]["Ivec"][t])
                other = sum(map(abs, [r.FF[(x,y)]["Ivec"][t], r.FF[(x,y)]["Svec"][t], r.FF[(x,y)]["Rvec"][t] , r.FF[(x,y)]["Evec"][t]]))
            cs.append(interested/ float(other))
    plt.scatter(xs, ys, c=cs, cmap="Reds", s=100)
    plt.title(str(r) + " " + constraint + " , t = " + str(t))

    if not os.path.isdir(gif_dir + r.filename() + "/" + constraint):
        os.makedirs(gif_dir + r.filename() + "/" + constraint)

    plt.savefig(gif_dir + r.filename() + "/" + constraint + "/{0:04}.png".format(t))
    plt.close()

inputs = range(30)
pool = mp.Pool(processes=4)
# results = pool.map(processInput, inputs)

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML, Image
from matplotlib.animation import FuncAnimation


def update_plot(i, data, scat):
    scat.set_array(data[i])
    return scat,


xys = np.asarray(list(product(range(14), range(14))))
xs = xys[:,0]
ys = xys[:,1]
numframes = 500
numpoints = len(xs)
color_data = np.random.random((numframes, numpoints))
cs = []

for t in range(numframes):
    cs = []
    for pt in list(product(range(14), range(14))):
        interested = abs(r.FF[pt]["Ivec"][t]) + abs(r.FF[pt]["Rvec"][t])
        other = sum(map(abs, [r.FF[pt]["Ivec"][t], r.FF[pt]["Svec"][t], r.FF[pt]["Rvec"][t] , r.FF[pt]["Evec"][t]]))
        cs += [interested/ float(other)]

    color_data[t,:] = np.asarray(list(cs))
# print color_data, len(color_data[0,:])
        

fig = plt.figure()
scat = plt.scatter(xs, ys, c=color_data[0,:], s=100, cmap="Reds")

ani = animation.FuncAnimation(fig, update_plot, frames=numframes,
                              fargs=(color_data, scat),blit=False)
plt.show()




In [ ]:
def load_arrays(data_dir,title):
    newcases = np.load(data_dir + "NEWCASE_" + title + ".npy")
    infected = np.load(data_dir + "INFECTED_" + title + ".npy") 
    infected_only = np.load(data_dir + "INFCONLY_" + title + ".npy")
    return newcases, infected, infected_only

In [ ]:

#     total_i = [np.zeros(len(r.FF[(0,0)]['Ivec'])) for x in range(4)]
#     total_e = [np.zeros(len(r.FF[(0,0)]['Ivec'])) for x in range(4)]
#     total_s = [np.zeros(len(r.FF[(0,0)]['Ivec'])) for x in range(4)]
#     total_r = [np.zeros(len(r.FF[(0,0)]['Ivec'])) for x in range(4)]
    
#     for x in xrange(14):
#         for y in xrange(14):
#             pair = (x,y)
            
#             total_i[0] += r.FF[pair]['Ivec']
#             total_i[1] += r.TT[pair]['Ivec']
#             total_i[2] += r.TF[pair]['Ivec']
#             total_i[3] += r.FT[pair]['Ivec']
# #             print total_i[0]
# #             print r.FF[pair]['Ivec']
# #             print "---------------------------"
# #             break
            
#             total_e[0] += r.FF[pair]['Evec']
#             total_e[1] += r.TT[pair]['Evec']
#             total_e[2] += r.TF[pair]['Evec']
#             total_e[3] += r.FT[pair]['Evec']
            
#             total_s[0] += r.FF[pair]['Svec']
#             total_s[1] += r.TT[pair]['Svec']
#             total_s[2] += r.TF[pair]['Svec']
#             total_s[3] += r.FT[pair]['Svec']
            
#             total_r[0] += r.FF[pair]['Rvec']
#             total_r[1] += r.TT[pair]['Rvec']
#             total_r[2] += r.TF[pair]['Rvec']
#             total_r[3] += r.FT[pair]['Rvec']
    
#     print total_i
       
    # Infected
    xs=np.arange(len(r.FF[pair]['Ivec']))
    plt.plot(xs, total_i[0], label="FF")
    plt.plot(xs, total_i[1], label="TT")
    plt.plot(xs, total_i[2], label="TF")
    plt.plot(xs, total_i[3], label="FT")
    plt.legend(0)
    plt.xlabel("Timestep")
    plt.ylabel("Number of Infected")
    plt.savefig(plot_dir + "_".join(str(r).split()) + "_I.png")
    plt.close()
    
# #     # Exposed
# #     plt.plot(xs, total_e[0], label="FF")
# #     plt.plot(xs, total_e[1], label="TT")
# #     plt.plot(xs, total_e[2], label="TF")
# #     plt.plot(xs, total_e[3], label="FT")
# #     plt.legend()
# #     plt.xlabel("Timestep")
# #     plt.ylabel("Number of Exposed")
# #     plt.savefig(plot_dir + "_".join(str(r).split()) + "_E.png")
# #     plt.close()
    
# #     # Susecptible
# #     plt.plot(xs, total_s[0], label="FF")
# #     plt.plot(xs, total_s[1], label="TT")
# #     plt.plot(xs, total_s[2], label="TF")
# #     plt.plot(xs, total_s[3], label="FT")
# #     plt.legend()
# #     plt.xlabel("Timestep")
# #     plt.ylabel("Number of Susceptible")
# #     plt.savefig(plot_dir + "_".join(str(r).split()) + "_S.png")
# #     plt.close()
    
# #     # Removed
# #     plt.plot(xs, total_r[0], label="FF")
# #     plt.plot(xs, total_r[1], label="TT")
# #     plt.plot(xs, total_r[2], label="TF")
# #     plt.plot(xs, total_r[3], label="FT")
# #     plt.legend()
# #     plt.xlabel("Timestep")
# #     plt.ylabel("Number of Removed")
# #     plt.savefig(plot_dir + "_".join(str(r).split()) + "_R.png")
# #     plt.close()

In [ ]:
# polyf = lambda x: coeff[0]*(x**2) + coeff[1]*(x) + coeff[2]
def polyf(x, coeff, d=2):
    """ Given the coefficients for a polynomial of degree 2, returns the value
        of the polynomial at position x. Generalized to any degree. """
    total = 0
    for i in range(d+1):
        total += coeff[i]*(x**(d-i))
    return total

def fit_polynomial(data, title, deg=2):
    coeff = np.polyfit(xrange(len(data)), data, deg)
    plt.plot(xrange(len(data)), data, label="og")
    plt.plot(xrange(len(data)), map(polyf, xrange(len(data))), label='poly fit')
    print polyf2(1), polyf(1)
    plt.legend(loc='best')
    plt.title(title)
    plt.show()

def fit_exponential(data, title):
    coeff = np.polyfit(xrange(len(data)), np.log(data), 1, w=np.sqrt(data))
    expf = lambda x: np.exp(coeff[1]) * np.exp(coeff[0]*x)
    plt.plot(xrange(len(data)), data, 'r-',label="og")
    plt.plot(xrange(len(data)), map(expf, xrange(len(data))), label='exp fit')
    plt.legend(loc='best')
    plt.title(title)
    plt.show()
    
def fit_both(data, title, deg=2, dotsize=1.6):
    # Correct for np.log(0)
    data += 1
    
    # Exponential fit
    exp_coeff = np.polyfit(xrange(len(data)), np.log(data), 1, w=np.sqrt(data))
    expf = lambda x: np.exp(exp_coeff[1]) * np.exp(exp_coeff[0]*x)
    exp_curve = map(expf, xrange(len(data)))
    
    # Polynomial fit
    poly_coeff = np.polyfit(xrange(len(data)), data, deg)
    poly_curve = [polyf(x, poly_coeff, deg) for x in xrange(len(data))]

    # Plot curves
    plt.plot(xrange(len(data)), exp_curve, label='$e^x$')
    plt.plot(xrange(len(data)), poly_curve, label='$x^'+str(deg)+'$')
    plt.plot(xrange(len(data)), data, 'ro', ms=dotsize, mew=0, label="data")
    plt.legend(loc='best')
    plt.title(title)
    plt.savefig("figs/" + title + ".pdf")
    plt.show()
    
    # Display R^2 statistics
    r1 = r_squared(data, exp_curve)
    r2 = r_squared(data, poly_curve)
    print "R-Squared (Exponential) =", r1
    print "R-Squared (Poly) =", r2
    print "Poly - Exponential =", r2 - r1
    if r1 > r2:
        print "Best fit is exponential."
    else:
        print "Best fit is polynomial."

def r_squared(data, fit):
    sst = ((data-data.mean())**2).sum()
    ssr = ((data - fit)**2).sum()
    return 1.0 - (float(ssr)/sst)
    
    

In [ ]:
# WHO Data
conkary_new_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/conkary_new.txt","r")
conkary_all_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/conkary_allinfected.txt","r")
SL_new_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/SL_new.txt","r")
SL_all_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/SL_cumulative.txt","r")
liberia_mon_new_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/liberia_mon_new.txt","r")
liberia_mon_all_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/liberia_mon_cumulative.txt","r")
port_loko_new_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/port_loko_new.txt","r")
port_loko_all_file = open("/Users/jonathanking/Dropbox/Documents/CMU-Pitt 2017/Coursework/Fall 17/CSM/Project/whodata/port_loko_cumulative.txt","r")

# Parse Data
conkary_new = np.asarray(map(int, conkary_new_file.read().split('\n')))
conkary_all = np.asarray(map(int, conkary_all_file.read().split('\n')))
SL_new = np.asarray(map(int, SL_new_file.read().split('\n')))
SL_all = np.asarray(map(int, SL_all_file.read().split('\n')))
liberia_mon_new = np.asarray(map(int, liberia_mon_new_file.read().split('\n')))
liberia_mon_all = np.asarray(map(int, liberia_mon_all_file.read().split('\n')))
port_loko_new = np.asarray(map(int, port_loko_new_file.read().split('\n')))
port_loko_all = np.asarray(map(int, port_loko_all_file.read().split('\n')))

# Plot data
fit_both(conkary_new, "Conkary New Cases")
fit_both(conkary_all, "Conkary Cumulative")
fit_both(SL_new, "Sierra Leone New Cases")
fit_both(SL_all, "Sierra Leone Cumulative")
fit_both(liberia_mon_new, "Montserrado, Liberia New Cases")
fit_both(liberia_mon_all, "Montserrado, Liberia Cumulative")
fit_both(port_loko_new, "Port Loko New Cases")
fit_both(port_loko_all, "Port Loko Cumulative")


In [ ]:
titles = ["size013_agents1600000_cmTrue__cnTrue__days365_t20171107_212506",
          "size013_agents1600000_cmFalse_cnTrue__days365_t20171107_144814",
          "size013_agents1600000_cmFalse_cnFalse_days365_t20171107_144801",
          "size013_agents1600000_cmTrue__cnFalse_days365_t20171103_160828",
          "size013_agents1600000_cmTrue__cnFalse_days1000_t20171102_154156",
          "size013_agents1600000_cmFalse_cnFalse_days1000_t20171102_154139",
          "size013_agents800000_cmFalse_cnFalse_days365_t20171112_180134_halfdense"]

conditions = ["cmTrue, cnTrue", 
              "cmFalse, cnTrue",
              "cmFalse cnFalse",
              "cmTrue cnFalse",
              "cmTrue cnFalse 1000days",
              "cmFalse cnFalse 1000days",
              "cmFalse cnFalse half-dense"]

for title, cond in zip(titles, conditions):
    new, infected, infected_only = load_arrays(data_dir, title)
    fit_both(infected, "Infected " + cond, deg=2, dotsize=1.1)


In [ ]:
for title, cond in zip(titles, conditions):
    new, infected, infected_only = load_arrays(data_dir, title)
    fit_both(new, "New Cases " + cond, deg=2)